All quotes below are excerpted from https://toadstyle.org/cryptopals/58.txt

```
58. Pollard's Method for Catching Kangaroos

The last problem was a little contrived. It only worked because I
helpfully foisted those broken group parameters on Alice and
Bob. While real-world groups may include some small subgroups, it's
improbable to find this many in a randomly generated group.

So what if we can only recover some fraction of the Bob's secret key?
It feels like there should be some way to use that knowledge to
recover the rest. And there is: Pollard's kangaroo algorithm.

This is a generic attack for computing a discrete logarithm (or
"index") known to lie within a certain contiguous range [a, b]. It has
a work factor approximately the square root of the size of the range.

The basic strategy is to try to find a collision between two
pseudorandom sequences of elements. One will start from an element of
known index, and the other will start from the element y whose index
we want to find.
```

In [1]:
from itertools import count
from random import randrange
from math import log

from challenge_31 import do_sha256, hmac
from challenge_39 import invmod
from challenge_57 import bob, crt, get_residues, primegen

In [2]:
p = 11470374874925275658116663507232161402086650258453896274534991676898999262641581519101074740642369848233294239851519212341844337347119899874391456329785623
q = 335062023296420808191071248367701059461
j = 34233586850807404623475048381328686211071196701374230492615844865929237417097514638999377942356150481334217896204702
g = 622952335333961296978159266084741085889881358738459939978290179936063635566740258555167783009058567397963466103140082647486611657350811560630587013183357

In [3]:
def f(y, k):
    return 2 ** (y % k)

In [4]:
def pollard(y, a, b, k=11, g=g, p=p, quiet=True):
    # implementation of Pollard's kangaroo algorithm. Follows the description given in 57.txt
    
    N = 4 * (2**k - 1) // k
    
    if not quiet:
        print(f"Starting Pollard's algorithm (k = {k}, N = {N})")
        print()
        print("Taking the tame kangaroo out for a walk.")
        
    xT = 0
    yT = pow(g, b, p)
    for i in range(N):
        f_yT = f(yT, k)  # precompute this value since we use it twice
        xT += f_yT
        yT = (yT * pow(g, f_yT, p)) % p

    if not quiet: print("Releasing the wild kangaroo! (this may take a while)")

    xW = 0
    yW = y
    while xW < b - a + xT:
        f_yW = f(yW, k)
        xW += f_yW
        yW = (yW * pow(g, f_yW, p)) % p

        if yW == yT:
            if not quiet: print("The wild kangaroo found something!")
            return b + xT - xW

    print("Didn't find anything this time...")  # the algorithm is probabilistic so this may happen

```
Implement Pollard's kangaroo algorithm. Here are some (less
accommodating) group parameters:

    p = 11470374874925275658116663507232161402086650258453896274534991676898999262641581519101074740642369848233294239851519212341844337347119899874391456329785623
    q = 335062023296420808191071248367701059461
    j = 34233586850807404623475048381328686211071196701374230492615844865929237417097514638999377942356150481334217896204702
    g = 622952335333961296978159266084741085889881358738459939978290179936063635566740258555167783009058567397963466103140082647486611657350811560630587013183357

And here's a sample y:

    y = 7760073848032689505395005705677365876654629189298052775754597607446617558600394076764814236081991643094239886772481052254010323780165093955236429914607119

The index of y is in the range [0, 2^20]. Find it with the kangaroo
algorithm.
```

In [5]:
y1 = 7760073848032689505395005705677365876654629189298052775754597607446617558600394076764814236081991643094239886772481052254010323780165093955236429914607119

a1, b1 = 0, 2**20
print("Recovering y1...")
result = pollard(y1, a1, b1, quiet=False)
print("Result:", result)
assert pow(g, result, p) == y1
print("It worked!")

Recovering y1...
Starting Pollard's algorithm (k = 11, N = 744)

Taking the tame kangaroo out for a walk.
Releasing the wild kangaroo! (this may take a while)
The wild kangaroo found something!
Result: 705485
It worked!


```
Wait, that's small enough to brute force. Here's one whose index is in
[0, 2^40]:

    y = 9388897478013399550694114614498790691034187453089355259602614074132918843899833277397448144245883225611726912025846772975325932794909655215329941809013733

Find that one, too. It might take a couple minutes.
```

In [6]:
y2 = 9388897478013399550694114614498790691034187453089355259602614074132918843899833277397448144245883225611726912025846772975325932794909655215329941809013733

a2, b2 = 0, 2**40
print("Recovering y2...")
result = pollard(y2, a2, b2, k=23, quiet=False)
print("Result:", result)
assert pow(g, result, p) == y2
print("It worked!")

Recovering y2...
Starting Pollard's algorithm (k = 23, N = 1458888)

Taking the tame kangaroo out for a walk.
Releasing the wild kangaroo! (this may take a while)
The wild kangaroo found something!
Result: 359579674340
It worked!


```
    ~~ later ~~

Enough about kangaroos, let's get back to Bob. Suppose we know Bob's
secret key x = n mod r for some r < q. It's actually not totally
obvious how to apply this algorithm to get the rest! Because we only
have:

    x = n mod r

Which means:

    x = n + m*r

For some unknown m. This relation defines a set of values that are
spread out at intervals of r, but Pollard's kangaroo requires a
continuous range!

Actually, this isn't a big deal. Because check it out - we can just
apply the following transformations:

    x = n + m*r
    y = g^x = g^(n + m*r)
    y = g^n * g^(m*r)
    y' = y * g^-n = g^(m*r)
    g' = g^r
    y' = (g')^m

Now simply search for the index m of y' to the base element g'. Notice
that we have a rough bound for m: [0, (q-1)/r]. After you find m, you
can plug it into your existing knowledge of x to recover the rest of
the secret.

Take the above group parameters and generate a key pair for Bob. Use
your subgroup-confinement attack from the last problem to recover as
much of Bob's secret as you can. You'll be able to get a good chunk of
it, but not the whole thing. Then use the kangaroo algorithm to run
down the remaining bits.
```

In [7]:
# This code was copied over from challenge_57.ipynb (since we can't import from Jupyter notebooks...)

# We'll implement Bob as a coroutine. He takes Diffie-Hellman public keys as inputs and yields (message, mac) pairs.
def bob_coro(message):
    x = randrange(0, q)
    x_pub = pow(g, x, p)  # public key - only yielded, not used
    print("Bob: Private key =", x)

    h = (yield x_pub)
    while True:
        secret = get_bytes(pow(h, x, p))
        K = do_sha256(secret)
        t = hmac(K, message)
        h = (yield (message, t))


assert log(p, 2) < 64*8
def get_bytes(n):
    # helper function: converts ints to bytes so we can hash them
    # depends on the assumption that n will fit into 64 bytes (asserted above)
    return n.to_bytes(64, 'big')


# helper function: generates primes using a modified, unbounded version of the Sieve of Eratosthenes
def primegen(up_to=None):
    yield 2
    d = {}
    counter = count(3, 2) if up_to is None else range(3, up_to, 2)
    for i in counter:
        l = d.pop(i, None)
        if l:
            for n in l:
                d.setdefault(i+(2*n), []).append(n)
            continue
        yield i
        d.setdefault(3*i, []).append(i)

In [8]:
print("Initializing Bob.")
bob = bob_coro(b"i know you've just about mcfuckin' had it, this shit is just magic")
bob_pub = next(bob)  # prints private key for reference

print("Launching Diffie-Hellman subgroup confinement attack.")

print("Partially factoring j...")
j_factors = [p for p in primegen(up_to=2**16) if j % p == 0 and (j // p) % p != 0]

print("Extracting residues from Bob. (😏)")
residues = get_residues(bob, j_factors, p=p, quiet=False)

print("Constraining Bob's private key using the CRT.")
residue, modulus = crt(residues, j_factors)
print("Bob's private key is congruent to", residue, "mod", modulus)

Initializing Bob.
Bob: Private key = 276179308142963894589145870386339283343
Launching Diffie-Hellman subgroup confinement attack.
Partially factoring j...
Extracting residues from Bob. (😏)
r = 2 ... Done.
r = 12457 ... Done.
r = 14741 ... Done.
r = 18061 ... Done.
r = 31193 ... Done.
r = 33941 ... Done.
r = 63803 ... Done.
Constraining Bob's private key using the CRT.
Bob's private key is congruent to 75075700418337208353518845 mod 448058868191464583449381646


In [9]:
# Apply the transformations from the quoted block above.
# This'll give us something we can use pollard() on.
# The names in this block may seem confusing; they were chosen for consistency with 58.txt's description.

y = bob_pub
n, r = residue, modulus

g_prime = pow(g, r, p)
g_inv = invmod(g, p)
y_prime = (y * pow(g_inv, n, p)) % p

# y_prime equals g_prime raised to some index m.
# m is also the coefficient in x = n + m*r.
# Once we find m, we can use this equation to find the private key x.
# We have a rough bound for m: [0, (q-1)/r]. This is enough to use pollard().

lb, ub = 0, (q-1) // r

# pollard() may fail, so let's call it in a loop and break as soon as it succeeds
for k in count(21, 2):
    print(f"Trying k={k}")
    m = pollard(y_prime, lb, ub, k, g_prime)
    if m is not None:
        break

x = n + m*r

print("Done! m =", m)
print("Bob's private key is", x)
print("Make sure to check this value against the logged value in the previous code block.")

Trying k=21
Done! m = 616390674863
Bob's private key is 276179308142963894589145870386339283343
Make sure to check this value against the logged value in the previous code block.
